In [3]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np

In [4]:
df = pd.read_csv('IMDB.csv')
df = df.sample(500)
df.to_csv('data.csv', index=False)
df.head()

,review,sentiment
265,"This movie is horrible, but you have to see it...",positive
282,i love bad shark movies. i really do. i laugh ...,negative
239,"Anyone that is looking for an episode of ""Law ...",positive
419,"I didn't really expect much from ""The Night Li...",negative
174,Jack Webb is riveting as a Marine Corp drill i...,positive


In [7]:
# data preprocessing

# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['review'] = df['review'].apply(lower_case)
        df['review'] = df['review'].apply(remove_stop_words)
        df['review'] = df['review'].apply(removing_numbers)
        df['review'] = df['review'].apply(removing_punctuations)
        df['review'] = df['review'].apply(removing_urls)
        df['review'] = df['review'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

In [8]:
df = normalize_text(df)
df.head()

,review,sentiment
265,movie horrible see discus entire film greatest...,positive
282,love bad shark movie really laugh hysterically...,negative
239,anyone looking episode law order csi would loo...,positive
419,really expect much night listener actually nev...,negative
174,jack webb riveting marine corp drill instructo...,positive


In [9]:
df['sentiment'].value_counts()

sentiment
negative    258
positive    242
Name: count, dtype: int64

In [10]:
x = df['sentiment'].isin(['positive','negative'])
df = df[x]

In [11]:
df['sentiment'] = df['sentiment'].map({'positive':1, 'negative':0})
df.head()

,review,sentiment
265,movie horrible see discus entire film greatest...,1
282,love bad shark movie really laugh hysterically...,0
239,anyone looking episode law order csi would loo...,1
419,really expect much night listener actually nev...,0
174,jack webb riveting marine corp drill instructo...,1


In [12]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [13]:
vectorizer = CountVectorizer(max_features=100)
X = vectorizer.fit_transform(df['review'])
y = df['sentiment']

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [16]:
import dagshub

mlflow.set_tracking_uri("https://dagshub.com/Zaeem-Hassan/Capstone-Project.mlflow")
dagshub.init(repo_owner='Zaeem-Hassan', repo_name='Capstone-Project', mlflow=True)

mlflow.set_experiment("Capstone-Project-1")

Initialized MLflow to track repo "Zaeem-Hassan/Capstone-Project"

Repository Zaeem-Hassan/Capstone-Project initialized!

2026/01/15 02:24:28 INFO mlflow.tracking.fluent: Experiment with name 'Capstone-Project-1' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/98119d05939f49bdb27cf4d64594e093', creation_time=1768425868799, experiment_id='0', last_update_time=1768425868799, lifecycle_stage='active', name='Capstone-Project-1', tags={}>

In [17]:
import mlflow
import logging
import os
import time
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

logging.info("Starting MLflow run...")

with mlflow.start_run():
    start_time = time.time()
    
    try:
        logging.info("Logging preprocessing parameters...")
        mlflow.log_param("vectorizer", "Bag of Words")
        mlflow.log_param("num_features", 100)
        mlflow.log_param("test_size", 0.25)

        logging.info("Initializing Logistic Regression model...")
        model = LogisticRegression(max_iter=1000)  # Increase max_iter to prevent non-convergence issues

        logging.info("Fitting the model...")
        model.fit(X_train, y_train)
        logging.info("Model training complete.")

        logging.info("Logging model parameters...")
        mlflow.log_param("model", "Logistic Regression")

        logging.info("Making predictions...")
        y_pred = model.predict(X_test)

        logging.info("Calculating evaluation metrics...")
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        logging.info("Logging evaluation metrics...")
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)

        logging.info("Saving and logging the model...")
        mlflow.sklearn.log_model(model, "model")

        # Log execution time
        end_time = time.time()
        logging.info(f"Model training and logging completed in {end_time - start_time:.2f} seconds.")

        # Save and log the notebook
        # notebook_path = "exp1_baseline_model.ipynb"
        # logging.info("Executing Jupyter Notebook. This may take a while...")
        # os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
        # mlflow.log_artifact(notebook_path)

        # logging.info("Notebook execution and logging complete.")

        # Print the results for verification
        logging.info(f"Accuracy: {accuracy}")
        logging.info(f"Precision: {precision}")
        logging.info(f"Recall: {recall}")
        logging.info(f"F1 Score: {f1}")

    except Exception as e:
        logging.error(f"An error occurred: {e}", exc_info=True)

2026-01-15 02:35:45,809 - INFO - Starting MLflow run...
2026-01-15 02:35:50,900 - INFO - Logging preprocessing parameters...
2026-01-15 02:35:51,986 - INFO - Initializing Logistic Regression model...
2026-01-15 02:35:52,003 - INFO - Fitting the model...
2026-01-15 02:35:52,731 - INFO - Model training complete.
2026-01-15 02:35:52,731 - INFO - Logging model parameters...
2026-01-15 02:35:53,075 - INFO - Making predictions...
2026-01-15 02:35:53,092 - INFO - Calculating evaluation metrics...
2026-01-15 02:35:53,195 - INFO - Logging evaluation metrics...
2026-01-15 02:35:54,730 - INFO - Saving and logging the model...
2026/01/15 02:35:54 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2026/01/15 02:36:23 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2026-01-15 02:36:27,091 - INFO - Model training and logging completed in 36.19 seconds.
2026-

🏃 View run mysterious-carp-353 at: https://dagshub.com/Zaeem-Hassan/Capstone-Project.mlflow/#/experiments/0/runs/754d32a56d8f4522b15b4e4618888cd5
🧪 View experiment at: https://dagshub.com/Zaeem-Hassan/Capstone-Project.mlflow/#/experiments/0
